In [1]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.applications.resnet50 import ResNet50
from keras.models import load_model
import tensorflow as tf
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd
from keras.utils import plot_model

Using TensorFlow backend.


In [2]:
train_path = 'X:/Flipkart GRID - Object Localization (Medium Complexity Round)/orig_data/training_images'

img_width, img_height = 640., 480.

In [3]:
train_df = pd.read_csv(os.path.join('X:/Flipkart GRID - Object Localization (Medium Complexity Round)/orig_data', 'training.csv'))
im_list = train_df.image_name.tolist()

In [4]:
print(len(im_list))
print(im_list[:5])
train_df.head()

14000
['JPEG_20160706_121146_1000145715002.png', 'JPEG_20161119_174038_1000690577600.png', '147444927651111470309333776-Roadster-Men-Casual-Shoes-3041470309333487-1.png', '147772332675720161028_161611.png', '1473315333651DeeplearnS11638.png']


image_name   x1   x2   y1   y2
0             JPEG_20160706_121146_1000145715002.png  115  495  143  325
1             JPEG_20161119_174038_1000690577600.png   23  457   61  409
2  147444927651111470309333776-Roadster-Men-Casua...   37  601   13  470
3                   147772332675720161028_161611.png   27  602  162  385
4                   1473315333651DeeplearnS11638.png   60  586  174  325

In [5]:
##### img_width, img_height = 640, 480
base_dir = 'X:/Flipkart GRID - Object Localization (Medium Complexity Round)/'
# train_data_dir = "X:/train-scene classification/data/train/"
# validation_data_dir = "X:/train-scene classification/data/valid/"
# test_data_dir = "X:/train-scene classification/data/test/"
epochs = 1000
# Save the model according to the conditions  
chkpt_dir = "X:/Flipkart GRID - Object Localization (Medium Complexity Round)/chkpts/resnet50model.h5" # Change for every new training session 

In [6]:
# base_model = ResNet50(weights = "imagenet", include_top=False, input_shape = (img_height, img_width, 3), classes = 4)

In [7]:
# base_model.save('resnet50.h5')
base_model = load_model(os.path.join(base_dir,'base_models', 'resnet50.h5'))

C:\Users\praty\Anaconda3\envs\dlenv\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [8]:
# for layer in base_model.layers:
#     layer.trainable = False
x = base_model.output
x = Flatten()(x)
# x = Dense(2048, activation="relu")(x)
# x = Dropout(0.7)(x)
x = Dense(32, activation="relu")(x)
x = Dense(32, activation="relu")(x)
# x = Dense(1024, activation="relu")(x)
# x = Dropout(0.5)(x)
# x = Dense(512, activation="relu")(x)
# x = Dropout(0.3)(x)
# predictions = Dense(6, activation="softmax")(x)
predictions = Dense(4, activation="linear")(x)

In [9]:
model = Model(input = base_model.input, output = predictions)
for layer in model.layers[:-3]:
    layer.trainable = False
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 480, 640, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 486, 646, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 240, 320, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 240, 320, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

C:\Users\praty\Anaconda3\envs\dlenv\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  """Entry point for launching an IPython kernel.


In [10]:
def mIoU(labels,predictions):
#     print(labels.shape)
#     print(predictions.shape)
#     print(labels[0])
#     print(predictions[0])
    pred_x1=predictions[:,0]#*img_width
    pred_x2=predictions[:,1]#*img_width
    pred_y1=predictions[:,2]#*img_height
    pred_y2=predictions[:,3]#*img_height
    label_x1=labels[:,0]#*img_width
    label_x2=labels[:,1]#*img_width
    label_y1=labels[:,2]#*img_height
    label_y2=labels[:,3]#*img_height
    int_x1 = tf.maximum(pred_x1, label_x1)
    int_x2 = tf.minimum(pred_x2, label_x2)
    int_y1 = tf.maximum(pred_y1, label_y1)
    int_y2 = tf.minimum(pred_y2, label_y2)
    tensor_type = pred_x1.dtype
    pred_area = tf.multiply(tf.maximum(tf.cast(0.0, tensor_type), pred_x2 - pred_x1), tf.maximum(tf.cast(0.0, tensor_type), pred_y2 - pred_y1))
    label_area = tf.multiply(tf.maximum(tf.cast(0.0, tensor_type), label_x2 - label_x1), tf.maximum(tf.cast(0.0, tensor_type), label_y2 - label_y1))
    int_area = tf.multiply(tf.maximum(tf.cast(0.0, tensor_type), int_x2 - int_x1), tf.maximum(tf.cast(0.0, tensor_type), int_y2 - int_y1))
    union_area = pred_area + label_area - int_area
    IoU = int_area / union_area
    return tf.reduce_mean(IoU)

In [11]:
# IoU Metric Test
LABEL = [[0.0, 1.0, 0.0, 1.0], 
        [2.0, 3.0, 2.0, 3.0]]
PRED = [[0.5, 1.5, 0.5, 1.5],
        [2.0, 3.0, 2.0, 3.0]]
LABEL = tf.constant(np.array(LABEL))
PRED = tf.constant(np.array(PRED))
print(mIoU(LABEL, PRED))
with tf.Session() as sess:
    val = sess.run(mIoU(LABEL, PRED))
    print(val)

Tensor("Mean:0", shape=(), dtype=float64)
0.5714285714285714


In [21]:
model.load_weights(chkpt_dir)

In [22]:
model.compile(loss = "mse", optimizer = optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-8), metrics=['mse', mIoU])

In [23]:
checkpoint = ModelCheckpoint(chkpt_dir, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_loss', min_delta=0, patience=40, verbose=1, mode='auto')

In [24]:
train_df = pd.read_csv(os.path.join(base_dir, 'data', 'train.csv'))
val_df = pd.read_csv(os.path.join(os.path.join(base_dir, 'data', 'val.csv')))

In [25]:
l = []
for im in val_df.image_name:
    img = cv2.imread(os.path.join(base_dir, 'data', 'validate', im))
    l.append(np.expand_dims(img, axis = 0))
x_val = np.vstack(l)
y_val = val_df.iloc[:,1:].values.astype(np.float64)

y_val[:, 0] = y_val[:, 0]# / img_width
y_val[:, 1] = y_val[:, 1]# / img_width
y_val[:, 2] = y_val[:, 2]# / img_height
y_val[:, 3] = y_val[:, 3]# / img_height
print(y_val[0])
print(y_val.dtype)
print(x_val.shape)

[  9. 636. 127. 279.]
float64
(1000, 480, 640, 3)


In [26]:
# model.load_weights(chkpt_dir)

In [ ]:
for epoch in range(epochs):
    for i in range(13):
        l = []
        for im in train_df.image_name[i * 1000: (i + 1) * 1000]:
            img = cv2.imread(os.path.join(base_dir, 'data', 'train', im))
            l.append(np.expand_dims(img, axis = 0))
        x_train = np.vstack(l)
        y_train = train_df[i * 1000: (i + 1) * 1000].iloc[:,1:].values
        y_train[:, 0] = y_train[:, 0]# / img_width
        y_train[:, 1] = y_train[:, 1]# / img_width
        y_train[:, 2] = y_train[:, 2]# / img_height
        y_train[:, 3] = y_train[:, 3]# / img_height
        
        model.fit(x=x_train, y=y_train, batch_size=32, epochs=1, verbose=1, callbacks=[checkpoint], validation_split=0.0, validation_data=(x_val, y_val), shuffle=True)

Train on 1000 samples, validate on 1000 samples
Epoch 1/1
1000/1000 [==============================] - 40s 40ms/step - loss: 345.1500 - mean_squared_error: 345.1500 - mIoU: 0.8562 - val_loss: 4583.0334 - val_mean_squared_error: 4583.0334 - val_mIoU: 0.5005

Epoch 00001: val_loss improved from inf to 4583.03343, saving model to X:/Flipkart GRID - Object Localization (Medium Complexity Round)/chkpts/resnet50model.h5
Train on 1000 samples, validate on 1000 samples
Epoch 1/1
1000/1000 [==============================] - 39s 39ms/step - loss: 206.4808 - mean_squared_error: 206.4808 - mIoU: 0.8806 - val_loss: 4303.2990 - val_mean_squared_error: 4303.2990 - val_mIoU: 0.5154

Epoch 00001: val_loss improved from 4583.03343 to 4303.29897, saving model to X:/Flipkart GRID - Object Localization (Medium Complexity Round)/chkpts/resnet50model.h5
Train on 1000 samples, validate on 1000 samples
Epoch 1/1
1000/1000 [==============================] - 39s 39ms/step - loss: 189.5350 - mean_squared_error: 1

1000/1000 [==============================] - 39s 39ms/step - loss: 163.0372 - mean_squared_error: 163.0372 - mIoU: 0.8873 - val_loss: 3932.8996 - val_mean_squared_error: 3932.8996 - val_mIoU: 0.5367

Epoch 00001: val_loss did not improve from 3832.58564
Train on 1000 samples, validate on 1000 samples
Epoch 1/1
1000/1000 [==============================] - 39s 39ms/step - loss: 194.1891 - mean_squared_error: 194.1891 - mIoU: 0.8797 - val_loss: 3967.2579 - val_mean_squared_error: 3967.2579 - val_mIoU: 0.5347

Epoch 00001: val_loss did not improve from 3832.58564
Train on 1000 samples, validate on 1000 samples
Epoch 1/1
1000/1000 [==============================] - 39s 39ms/step - loss: 139.7719 - mean_squared_error: 139.7719 - mIoU: 0.8981 - val_loss: 4204.0221 - val_mean_squared_error: 4204.0221 - val_mIoU: 0.5213

Epoch 00001: val_loss did not improve from 3832.58564
Train on 1000 samples, validate on 1000 samples
Epoch 1/1
1000/1000 [==============================] - 39s 39ms/step - los

1000/1000 [==============================] - 39s 39ms/step - loss: 149.8927 - mean_squared_error: 149.8927 - mIoU: 0.8970 - val_loss: 3738.5071 - val_mean_squared_error: 3738.5071 - val_mIoU: 0.5494

Epoch 00001: val_loss did not improve from 3497.45220
Train on 1000 samples, validate on 1000 samples
Epoch 1/1
1000/1000 [==============================] - 39s 39ms/step - loss: 103.8956 - mean_squared_error: 103.8956 - mIoU: 0.9078 - val_loss: 3912.0092 - val_mean_squared_error: 3912.0092 - val_mIoU: 0.5355

Epoch 00001: val_loss did not improve from 3497.45220
Train on 1000 samples, validate on 1000 samples
Epoch 1/1
1000/1000 [==============================] - 39s 39ms/step - loss: 150.7615 - mean_squared_error: 150.7615 - mIoU: 0.8925 - val_loss: 3584.0414 - val_mean_squared_error: 3584.0414 - val_mIoU: 0.5552

Epoch 00001: val_loss did not improve from 3497.45220
Train on 1000 samples, validate on 1000 samples
Epoch 1/1
1000/1000 [==============================] - 39s 39ms/step - los

In [18]:
model.save('transfer_resnet50.h5')

In [20]:
model.load_weights(chkpt_dir)
LABEL = y_val
PRED = model.predict(x_val)
with tf.Session() as sess:
    val = sess.run(mIoU(LABEL, PRED))
    print(val)

0.5942009
